In [1]:
#REFS:
#Short presentations for alternating and symmetric groups, J.N. Bray, M.D.E. Conder, C.R. Leedham-Green, and E.A. O’Brien
#https://en.wikipedia.org/wiki/Chinese_remainder_theorem#Generalization_to_arbitrary_rings

In [2]:
#we can represent the group algebra of a cyclic group as a polynomial ring quotient an ideal 
#F[C_N] = F[x]/(x^N-1)
#and factor x^N-1, and use the Chinese remainder theorem and Bezout's identity for the isomorphism.
#Similarly, we can write down a presentation of S_N, <\sigma, \tau | \sigma^n = 1, \tau^2 = 1, ...>,
#with just two generators and then the group algebra is a quotient of a free algebra on two generators,
#F[S_N] = F<x,y>/ (x^2 = y^n = (xy)^{n−1} = 1, (xy^{−1}xy)^3 = 1, (xy^{−j}xy^j)^2 = 1 for 2 \le j \le ⌊n/2⌋)
#We still have the Chinese remainder theorem, and there algorthims to factor non-commutative polynomials. 
#It turns out this is equivalent to using central orthogonal idempotents.
#Question: Do we gain anything with this approach?

In [3]:
#Let I = I_1 \cap \ldots \cap I_k be the intersection of pairwise coprime two-sided ideals, 
#i.e. there exists i+j = 1 in each distinct pair I_i, I_j.
#Let phi: F<x,y>/I --> F<x,y>/I_1 \times \ldots \times F<x,y>/I_k
#Let f_i = (0,0, \ldots , 1 , \ldots 0, 0), a 1 in th i-th component and 0's elsewhere.
#The e_i = \phi^-1(f_i). I_i = F<x,y>(1-e_i).
#In this case we have the idempotents e_i = e_i(x,y), so we can construct the ideals I_i = F<x,y>(1-e_i). 
#So really we just need to compute 1-e_i(x,y), so convert the group elements their words in x, y.

In [161]:
#a short presentation for S_n
#{x,y|x^2 = y^n = (xy)^{n−1} = 1, (xy^{−1}xy)^3 = 1, (xy^{−j}xy^j)^2 = 1 for 2 \le j \le ⌊n/2⌋}
n=4
F.<a,b> = FreeGroup()
G = F / ([a^2, b^n, (a*b)^(n-1), (a*b^(-1)*a*b)^3] + [(a*b^(-j)*a*b^j)^2 for j in range(2,floor(n/2)+1)])
G.is_isomorphic(SymmetricGroup(n))

True

In [210]:
#CONSTRUCT EXPLICIT ISOMORPHISM 
#obtain map from quotient of free group to symmetric group using GAP library
#[a^2, b^n, (a*b)^(n-1), (a*b^(-1)*a*b)^3] + [(a*b^(-j)*a*b^j)^2 for j in range(2,floor(n/2)+1)]
#BUG: getting different values every time
#FIX: that's okay, just precompute this, don't compute it every time
def symmetric_group_gens(n):
    f=gap.FreeGroup(2)
    relations = gap([f.1^2, f.2^n, (f.1*f.2)^(n-1), (f.1*f.2^(-1)*f.1*f.2)^3] + [(f.1*f.2^(-j)*f.1*f.2^j)^2 for j in range(2,floor(n/2)+1)])
    g = f / relations
    Sn = gap.SymmetricGroup(n)
    isom = gap.IsomorphismGroups(g,S4)
    im_g = gap.Image(isom)
    sym_gens = gap.GeneratorsOfGroup(im_g)
    return list(sym_gens)

In [174]:
SymmetricGroup(n)(symmetric_group_gens(4)[0])

(1,3)

In [6]:
#we can compute the homset, but I can't iterate through it!
G_to_sym_homs = Hom(G,SymmetricGroup(n))

In [7]:
type(G_to_sym_homs)

<class 'sage.groups.libgap_morphism.GroupHomset_libgap_with_category'>

In [8]:
#we can easily and quickly compute idempotents for the symmetric group algebra
p=3;
SGA_GFp_n = SymmetricGroupAlgebra(GF(p),n)
idems = SGA_GFp_n.central_orthogonal_idempotents()
#for each group element in idempotents, find expression as a word in x,y
idems[2]*idems[2] == idems[2]

True

In [9]:
#can use free group algebra corresponding to G
R = GF(3)
FGA = GroupAlgebra(G, R); FGA

Algebra of Finitely presented group < a, b | a^2, b^4, (a*b)^3, (a*b^-1*a*b)^3, (a*b^-2*a*b^2)^2 > over Finite Field of size 3

In [182]:
#map from symmetric group algebra to quotient of free group algebra
#we must use the standard symmetric group in order to solve the word problem
#TO-DO: need to map solved word problem back to G
#need to ensure isomorphism G <--> S_n is implemented properly
def SGA_to_FGA_quotient(v):
    #solve word problem for each basis element
    FGA_convert = 0
    sym_gens = symmetric_group_gens(n)
    for pair in list(v):
        coeff = FGA(pair[1])
        sigma = SymmetricGroup(n)(pair[0])
        #TO-DO: these should be replaced with appropriate generators from the isomorphim G <--> S_n
        gens = [SymmetricGroup(n)(sym_gens[0]), SymmetricGroup(n)(sym_gens[1])]
        word_gens, word_sym = sigma.word_problem(gens,display=False)
        print(word_gens,word_sym)
        #handle identity case separately
        if word_gens == "<identity ...>":
            replace_with_free_group_gens = "1"
        else:
            #map word in generators to free group quotient
            #need to ensure this map is actually an homomorphism
            #x2 <--> (1 2) <--> a, x1 <--> (1 2 ... n) <--> b
            replace_with_free_group_gens = word_gens.replace("x2","a").replace("x1","b")
        word_to_free_group_quotient = sage_eval(replace_with_free_group_gens,locals={'a':G.gens()[0],'b':G.gens()[1]})
        FGA_convert += coeff*FGA(word_to_free_group_quotient)
    return FGA_convert

In [183]:
poly1 = SGA_to_FGA_quotient(idems[0]); poly1

[(2,3), (1,2,3,4)]
x1^-1*x2*x1^-1*x2^-2 (2,3)^-1*(1,2,3,4)*(2,3)^-1*(1,2,3,4)^-2
[(2,4), (1,3,2,4)]
x2^-1*(x2^-1*x1^-1)^2*x2*x1^-1*x2^-2*x1^-1*x2 (1,3,2,4)^-1*((1,3,2,4)^-1*(2,4)^-1)^2*(1,3,2,4)*(2,4)^-1*(1,3,2,4)^-2*(2,4)^-1*(1,3,2,4)
[(1,3), (1,3,4,2)]
(x1^-1*x2^-1)^2*x2^-1*x1^-1*x2 ((1,3)^-1*(1,3,4,2)^-1)^2*(1,3,4,2)^-1*(1,3)^-1*(1,3,4,2)
[(2,4), (1,4,2,3)]
x2^-1*x1^-1*x2^-2*x1^-1 (1,4,2,3)^-1*(2,4)^-1*(1,4,2,3)^-2*(2,4)^-1
[(2,4), (1,3,4,2)]
x1^-1*(x2^-2*x1^-1*x2^-1)^2*x1^-1*x2^-1 (2,4)^-1*((1,3,4,2)^-2*(2,4)^-1*(1,3,4,2)^-1)^2*(2,4)^-1*(1,3,4,2)^-1
[(1,3), (1,4,2,3)]
x1^-1*x2*x1^-1*x2^-1*x1^-2*x2^-1 (1,3)^-1*(1,4,2,3)*(1,3)^-1*(1,4,2,3)^-1*(1,3)^-2*(1,4,2,3)^-1
[(2,3), (1,2,3,4)]
x2*x1^-1*x2^-1*x1^-1*x2*x1^-1*x2^-2 (1,2,3,4)*(2,3)^-1*(1,2,3,4)^-1*(2,3)^-1*(1,2,3,4)*(2,3)^-1*(1,2,3,4)^-2
[(1,3), (1,3,2,4)]
(x1^-1*x2^-2)^2 ((1,3)^-1*(1,3,2,4)^-2)^2
[(3,4), (1,2,3,4)]
x1^-1*x2^-2*x1^-1 (3,4)^-1*(1,2,3,4)^-2*(3,4)^-1
[(1,4), (1,4,3,2)]
x2^-1 (1,4,3,2)^-1
[(3,4), (1,3,4,2)]
x2^-1 (1,3,

a*b^-1*a^-1*b^-1*a*b^-1*a^-2 + 2*(b^-1*a^-1)^2*a^-1*b^-1*a + (b^-1*a^-2)^2 + b^-1*a^-2*b^-1 + 2*a^-1 + a + 2*a^-1*b^-1*a^-2*b^-1 + 2*b^-1*(a^-2*b^-1*a^-1)^2*b^-1*a^-1 + 2*b^-1*a*b^-1*a^-2 + b^-1*a^-1*b^-1 + 2*a^-1*(a^-1*b^-1)^2*a*b^-1*a^-2*b^-1*a + 2*b^-1*a*b^-1*a^-1*b^-2*a^-1 + 2*b*a*b

In [177]:
poly2 = SGA_to_FGA_quotient(idems[1]); poly2

a^-2 + (b^-1*a^-2)^2 + a^-1 + 2*a + b^-1*(a^-1*b^-1*a^-1)^2*b^-1 + b^-1 + 2*b^-1*a*b^-1*a^-2 + a*b^-1*a^-1 + 2*b*a*b + (b^-1*a^-2)^2*a^-1 + b^-1*a^-1*(a^-1*b^-1)^2*a + b^-1*a^-1*(a^-1*b^-1)^2*a^-1

In [178]:
poly3 = SGA_to_FGA_quotient(idems[2]); poly3

1 + b^-1*a*b^-1*a^-2*b^-2*a*b^-1*a^-1 + 2*(b^-1*a^-2)^2

In [179]:
#this should be idempotent
poly3*poly3

1 + (b^-1*a^-2)^4 + (b^-1*a*b^-1*a^-2*b^-2*a*b^-1*a^-1)^2 + 2*b^-1*a*b^-1*a^-2*b^-2*a*b^-1*a^-1 + (b^-1*a^-2)^2 + 2*(b^-1*a^-2)^2*b^-1*a*b^-1*a^-2*b^-2*a*b^-1*a^-1 + 2*b^-1*a*b^-1*a^-2*b^-2*a*b^-1*(a^-1*b^-1*a^-1)^2*a^-1

In [15]:
#could use Groebner bases for non-commutative polynomials to extract information from idempotent polynomials
#GAP has a package GBNP, https://www.gap-system.org/Packages/gbnp.html
#we can call Gap from Sage
#need to install GBNP package
#SAGE_ROOT = "/Applications/SageMath-10-3.app/Contents/Frameworks/sage.framework/Versions/10.3"
#GAP_PKG = "local/lib/gap/pkg"

In [16]:
gap('LoadPackage("gbnp")')

true

In [17]:
#gap('PrintNP([[[1,2],[2,1]],[3,-1]])') should yield 3ab - ba

In [18]:
#one could use new variables w=a^-1 and z=b^-1 to get rid of the inverses
#then one is factoring a polynomial of four variables